In [ ]:
### NEEDS pyCloudCompare.py in same directory as this .ipynb file ###

### BRIEF OVERVIEW
### 1. Takes input of RDLA lichen, turns it into RDLA with volume (represented 
### as a point cloud), but no meshing via distance mapping
### 2. Meshes the RDLA volume using ball pivoting for surface reconstruction
### 3. Performs dust accumulation to simulate the use of surface remote sensing
### such as laser scanning or photogrammetry on the RDLA mesh
### 4. Then samples the lowest z-value points, by placing a grid over the dust accumulation
### point cloud and calculating the index of the point in the dust accumulation with the
### smallest z-coordinate in each grid space.
### 5. Meshes these lowest z coordinate points by calculating normals for each point 
### and then using Poisson surface reconstruction. 
### 6. Simplifies this mesh into 1024 faces by using the quadratic edge collapse decimation
### algorithm.

### HISTOGRAM COMPARISON
### 1. Using open3d, calculates the distance between dust accumulation and seed mesh
### then samples each band in distances to match the histogram of the laser scan
### 2. Then prints the distribution as a histogram to test against the laser scan histogram

import os
import pymeshlab as ml
import numpy as np
import open3d as o3d
import pyCloudCompare as cc
import matplotlib.pyplot as plt
import matplotlib
import time
from time import strftime, localtime
import csv
from scipy.stats import linregress
from pathlib import Path
from scipy.stats import wilcoxon
from scipy.stats import spearmanr
from scipy.stats import chisquare
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

# Start timing
start_time = time.time()

mean_dist = []
std_dist = []
num_points = 0
num_points_array = []

# Set the input folder containing .obj meshes
input_folder = r"***********************"
# Set the output folder where the processed meshes will be saved
output_folder = r"***********************"
# Set the seed mesh folder for mesh to cloud distance calculation
seed_folder = r"***********************"

#distance map parameters:

# grid divison n x n x n
distanceMapGridstep = 0.0007
# Calculate the clearance value (e.g., 5 mm)
clearance = 0.01

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


def import_point_cloud_from_obj(obj_file_path):
    try:
        # Load the .obj file using open3d
        pcd = o3d.io.read_point_cloud(obj_file_path, format='ply')
        # Convert the point cloud data to a numpy array
        points = pcd.points

        # Convert numpy array to a list of tuples (x, y, z)
        point_cloud = [tuple(point) for point in points]

        return point_cloud
        #print(point_cloud)

    except Exception as e:
        print("Error importing point cloud:", e)
        return None

def divide_point_cloud_into_grid(point_cloud, grid_size=(45, 45)):
    # Get the bounding box of the point cloud
    min_bound = np.min(point_cloud, axis=0)
    max_bound = np.max(point_cloud, axis=0)
    
    # Calculate the size of each grid cell
    grid_size_x = (max_bound[0] - min_bound[0]) / grid_size[0]
    grid_size_y = (max_bound[1] - min_bound[1]) / grid_size[1]

    # Initialize an array to hold the points in each grid cell
    grid_cells = [[[] for _ in range(grid_size[1])] for _ in range(grid_size[0])]
    
    # Iterate through each point in the point cloud and find its grid cell
    for point in point_cloud:
        x_idx = int((point[0] - min_bound[0]) // grid_size_x)
        y_idx = int((point[1] - min_bound[1]) // grid_size_y)

        # Ensure the point is within the grid boundaries
        x_idx = min(grid_size[0] - 1, max(0, x_idx))
        y_idx = min(grid_size[1] - 1, max(0, y_idx))

        # Add the point to the corresponding grid cell
        grid_cells[x_idx][y_idx].append(point)

    return grid_cells

# Example usage:
# Replace "path/to/your_point_cloud.obj" with the actual path to your .obj file
# point_cloud = o3d.io.read_point_cloud("path/to/your_point_cloud.obj")
# if point_cloud is not None:
#     point_cloud_array = np.asarray(point_cloud.points)
#     grid_cells = divide_point_cloud_into_grid(point_cloud_array)

def calculate_minimum_in_grid_cells(grid_cells, grid_division=45):
    # Calculate the minimum value in each grid cell
    min_points = []
    for row in grid_cells:
        for cell in row:
            if cell:
                # Get the index of the point with the minimum z-coordinate value
                min_z_index = np.argmin([point[2] for point in cell])
                min_point = cell[min_z_index]
                min_points.append(min_point)
            else:
                # If a grid cell is empty, append a placeholder value (e.g., None)
                min_points.append(None)

    return min_points[:grid_division*grid_division]  # Return the first 1024 points (in case there are more than 1024 cells)

# Example usage:
# Assuming you have already imported and divided the point cloud into grid_cells
# min_values = calculate_minimum_in_grid_cells(grid_cells)

# MAXIMUM IS FOR CALCULATING THE HEIGHT OF THE RDLA LICHEN DISTANCE AWAY FROM THE SAMPLE MESH
# but we don't use it because we opt for calculating distance from samples mesh by point to mesh calculation
# using cloud compare
def calculate_maximum_in_grid_cells(grid_cells, grid_division=45):
    # Calculate the maximum value in each grid cell
    max_points = []
    for row in grid_cells:
        for cell in row:
            if cell:
                # Get the index of the point with the maximum z-coordinate value
                max_z_index = np.argmax([point[2] for point in cell])
                max_point = cell[max_z_index]
                max_points.append(max_point)
            else:
                # If a grid cell is empty, append a placeholder value (e.g., None)
                max_points.append(None)

    return max_points[:grid_division*grid_division]  # Return the first 1024 points (in case there are more than 1024 cells)

# Example usage:
# Assuming you have already imported and divided the point cloud into grid_cells
# max_values = calculate_maximum_in_grid_cells(grid_cells)

def save_points_to_ply_file(min_points, file_path):
    # Filter out the None elements from min_points
    valid_points = [point for point in min_points if point is not None]

    # Convert the list of points to a numpy array
    point_array = np.array(valid_points)

    # Create an Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_array)

    # Save the point cloud to a .ply file
    o3d.io.write_point_cloud(file_path, pcd)

# ENSURES SAMPLING HISTOGRAM FOLLOWS THAT OF THE LICHEN LASER SCAN 
def histogram_correction(mesh, dust_accumulation_file_name, file_path):
    # create a scene, put the seed mesh into the scene
    scene = o3d.t.geometry.RaycastingScene()
    _ = scene.add_triangles(mesh)
    
    # Resample dust accumulation points according to the laser scan distribution
    # get dust accumulation points
    query_points = np.float32(import_point_cloud_from_obj(dust_accumulation_file_name))
    
    #calculate distance from dust to seed mesh
    distance_dust = scene.compute_distance(query_points)
    distance_dust = distance_dust.numpy()  # Extract distances as a NumPy array
    
    # Sort points based on distances
    sorted_indices = np.argsort(distance_dust)  # Indices of points sorted by distance
    closest_points = query_points[sorted_indices]  # Points sorted by closest to farthest
    
    # define the bin widths in metre
    bins_mm = np.array([0.0, 0.00125, 0.00250, 0.00375, 0.00500, 0.00625, 0.00750, 0.00875, 0.01000, 0.01125,
                0.01250, 0.01375, 0.01500, 0.01625, 0.01750, 0.01875, 0.02000, 0.02125, 0.02250, 0.02375, 0.02500,
                0.02625, 0.02750, 0.02875, 0.03000, 0.03125, 0.03250, 0.03375, 0.03500, 0.03625, 0.03750])
    
    # calculate how many points fall in each bin 
    hist, bins = np.histogram(distance_dust, bins=bins_mm)
    
    # in case there are slices with 0 points
    for i, j in enumerate(hist):
        if hist[i] == 0:
            hist[i] = 1
    
    # calculate the area of the seed mesh and convert to number of points for sampling
    # 10 times more points than the RDLA from the c++ code
    # surface area to point number conversion
    mesh = o3d.io.read_triangle_mesh(os.path.join(seed_folder,"1024_F602.obj"))
    surface_area = mesh.get_surface_area()
    dust_population = (surface_area / (pow(0.01, 2) * np.pi)) * 2000
    
    # collect the points (as coordinates) into 2D array: bin_array
    lower_bin_boundary = 0
    bin_array = []
    for i in hist:
        bin_array.append(closest_points[lower_bin_boundary:(lower_bin_boundary + i)])
        lower_bin_boundary = lower_bin_boundary + i
    
    # collect the distance of laser scan lichen points from the stone 30 surface 
    mean_dist_array_lichen = []
    with open(r'***********************' + '.csv', newline='') as csvfile2:
        reader2 = csv.reader(csvfile2, delimiter=' ')
        for row in reader2:
            mean_dist_array_lichen.append(float(row[0]))
     
    # bin the laser scan lichen points, then calculate probability of these points of falling into the bins/bands
    hist_l, bins_l = np.histogram(mean_dist_array_lichen, bins=bins_mm, density=True)
    hist_lichen = hist_l*0.00125 # 0.00125 being the width of the bins in metre
    hist_lichen = dust_population * hist_lichen
    
    counter = 1
    new_dust_points = []
    
    # sample from bin_array which has coordinates of points in 2-D array
    # using accumulating hist_lichen/hist
    previous = 0
    for index1, value1 in enumerate(bin_array):
        counter = 1
        previous = 0
        for index2, value2 in enumerate(bin_array[index1]):
            min_bound = (hist_lichen[index1]/hist[index1]) * counter
            counter = counter + 1
            round_down = np.floor(min_bound)
            if round_down > (previous+0.5):
                new_dust_points.append(bin_array[index1][index2])
                previous = round_down
    
    # Convert the array to an Open3D point cloud
    pcd_adjust = o3d.geometry.PointCloud()
    pcd_adjust.points = o3d.utility.Vector3dVector(new_dust_points)
    
    # Save as a .ply file
    o3d.io.write_point_cloud(file_path, pcd_adjust)



#f = open('readme.txt', 'w') 
# Loop through all .obj files in the input folder
for file in sorted(Path(input_folder).iterdir(), key=os.path.getmtime):
    if file.parts[-1].endswith('.csv'): # .endswith(".csv"):

        #################################################################
        ### DISTANCE MAPPING
        
        # import point cloud (uses open3d)
        #point_cloud = import_point_cloud_from_obj(input_folder + '\\' + file)
        np_ptcld = np.genfromtxt(input_folder + '\\' + file.parts[-1], delimiter=',')
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(np_ptcld)
        
        # Get the minimum and maximum coordinates of the point cloud
        min_bound = np.asarray(pcd.get_min_bound())
        max_bound = np.asarray(pcd.get_max_bound())
        
        # Adjust the minimum and maximum bounds to create the bounding cube with clearance
        min_bound -= clearance
        max_bound += clearance
        
        # Calculate the number of points needed in each dimension of the grid
        num_points_x = int(np.ceil((max_bound[0] - min_bound[0]) / distanceMapGridstep))
        num_points_y = int(np.ceil((max_bound[1] - min_bound[1]) / distanceMapGridstep))
        num_points_z = int(np.ceil((max_bound[2] - min_bound[2]) / distanceMapGridstep))
        
        # Generate the query points grid using numpy.mgrid with the calculated number of points
        grid_x, grid_y, grid_z = np.mgrid[min_bound[0]:max_bound[0]:num_points_x*1j,
                                          min_bound[1]:max_bound[1]:num_points_y*1j,
                                          min_bound[2]:max_bound[2]:num_points_z*1j]
        
        # Combine the grid points into a single array of shape [num_points, 3]
        query_points = np.column_stack((grid_x.ravel(), grid_y.ravel(), grid_z.ravel())).astype(np.float32)

        # Convert the array to a PointCloud object in Open3D
        query_point_cloud = o3d.geometry.PointCloud()
        query_point_cloud.points = o3d.utility.Vector3dVector(query_points.reshape(-1, 3))
        
        # Calculate distances from the point cloud to the query points
        distances = o3d.geometry.PointCloud.compute_point_cloud_distance(query_point_cloud, pcd)
        
        # Convert distances to a NumPy array for reshaping and visualization
        distances_np = np.asarray(distances)
        
        # Reshape the distances_np array to match the shape of the query grid
        query_shape = (len(grid_x), len(grid_x[0]), len(grid_x[0][0]))
        distances_reshaped = distances_np.reshape(query_shape)
        
        # Define the set distance you want to find points at (e.g., 0.875 mm) for
        # 1.75 mm branch thickness. The 0.003 buffer zone around the shell of wanted
        # distance map values ensures enough points are obtained for the mesh reconstruction.
        set_distance_min = 0.000875 - 0.0003
        set_distance_max = 0.000875 + 0.0003
        #set_distance_min = 0
        #set_distance_max = 10
        
        # Find the indices of the query points that are approximately 1.5 mm away from the point cloud
        indices_dist_range = np.asarray(np.logical_and(np.abs(distances_np) >= set_distance_min,
                                               np.abs(distances_np) <= set_distance_max)).nonzero()
        
        # Retrieve the coordinates of the points that are approximately 1.5 mm away from the point cloud
        points_dist_range = np.asarray(query_point_cloud.points)[indices_dist_range]

        #save_points_to_ply_file(points_dist_range, output_folder + '\\' + file[5:9] + 'RDLA_' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
        #                        + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
        #                        + 'max' + str(set_distance_max)[4:] + '.ply')

        #################################################################
        ### BALL PIVOTING
        # create meshlab object
        ball_pivot_mesh = ml.MeshSet()

        # fill meshlab object with points from the extracted distance map
        vertices = np.asarray(points_dist_range)
        mesh_converted = ml.Mesh(vertices)
        ball_pivot_mesh.add_mesh(mesh_converted, "ball_pivoted_mesh")

        # if point cloud simplification is needed
        #ball_pivot_mesh.apply_filter("generate_simplified_point_cloud", radius = ml.AbsoluteValue(0.0008))
        
        # Apply the ball pivot algorithm for meshing distance map points
        ball_pivot_mesh.apply_filter("generate_surface_reconstruction_ball_pivoting", ballradius = ml.AbsoluteValue(0.0009), creasethr = 150)

        # close any remaining holes from the ball-pivoting
        ball_pivot_mesh.apply_filter("meshing_close_holes")

        # file name for ball pivot mesh output
        ball_pivot_file_name = os.path.join(output_folder, file.parts[-1][5:9] + 'RDLA_ball_recon' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        ball_pivot_mesh.save_current_mesh(ball_pivot_file_name)

        #################################################################
        # DUST ACCUMULATION

        # Apply the generate_dust_accumulation_point_cloud function
        ball_pivot_mesh.apply_filter("generate_dust_accumulation_point_cloud", dust_dir = [0,-0.70711,0.70711], nparticles = 3)

        dust_accumulation_file_name = os.path.join(output_folder, file.parts[-1][5:9] + 'RDLA_dust' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        ball_pivot_mesh.save_current_mesh(dust_accumulation_file_name)

        # DISTANCE BETWEEN DUST AND SEED MESH
        pcd = o3d.io.read_point_cloud(dust_accumulation_file_name, format='ply')
        mesh = o3d.io.read_triangle_mesh(os.path.join(seed_folder,"1024_F602.obj"))
        mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

        # perform histogram correction to ensure the dust accumulation (sampling)
        # follows the distribution of the laser scanned lichen data
        dust_accumulation_corrected_file_name = os.path.join(output_folder, file.parts[-1][5:9] + 'RDLA_dust' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                        + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                        + 'max' + str(set_distance_max)[4:] + '_corrected.ply')
        
        histogram_correction(mesh, dust_accumulation_file_name, dust_accumulation_corrected_file_name)
        
        #################################################################
        ### MIN Z GRID
        point_cloud = import_point_cloud_from_obj(dust_accumulation_file_name)
        
        grid_division = 50
        divided_ptcld = divide_point_cloud_into_grid(point_cloud, (grid_division,grid_division))
        final_cell = calculate_minimum_in_grid_cells(divided_ptcld, grid_division)
        #print(len(final_cell))

        min_z_file_name = os.path.join(output_folder, filepart + 'RDLA_lowest_z' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        save_points_to_ply_file(final_cell, min_z_file_name)
        
        #################################################################
        ### NORMALS CALCULATION

        # create meshlab object
        output_mesh = ml.MeshSet()

        # Filter out the None elements from min_points
        valid_points = [point for point in final_cell if point is not None]
        # Convert the list of points to a numpy array
        point_array = np.array(valid_points)
                
        # fill meshlab object with points from the extracted distance map
        vertices = np.asarray(point_array)
        mesh_converted = ml.Mesh(vertices)
        output_mesh.add_mesh(mesh_converted, "ball_pivoted_mesh")
        
        output_mesh.apply_filter("compute_normal_for_point_clouds", k = 6, flipflag = True, viewpos = [0, 0, 1])
        #################################################################
        ### SURFACE RECONSTUCTION SCREENED POISSON

        output_mesh.apply_filter("generate_surface_reconstruction_screened_poisson", scale = 1.0)
        
        #################################################################
        ### SIMPLIFICATION: QUADRATIC EDGE COLLAPSE DECIMATION: 1024
        output_mesh.apply_filter("meshing_decimation_quadric_edge_collapse", targetfacenum = 1024, qualitythr = 1.0, 
                                 preservenormal = True, preservetopology = True)
        
        final_1024_mesh = os.path.join(output_folder, filepart + 'RDLA_final_1024' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        output_mesh.save_current_mesh(final_1024_mesh)

        
        for file2 in os.listdir(seed_folder):
            if file2.endswith("1024_F602.obj"):
                cli = cc.CloudCompareCLI()
                cmd = cli.new_command()
                cmd.silent()  # Disable console
                cmd.open(seed_folder + '\\' + file2)  # Read file
                cmd.open(final_1024_mesh)  # Read file
                cmd.c2m_dist()
                #cmd.cloud_export_format(cc.CLOUD_EXPORT_FORMAT.ASCII, extension="csv")
                #cmd.save_clouds('GAMMA_CALCULATE_OUTPUT\\' + file.parts[-1][:27] + '.csv')
                print(cmd)
                cmd.execute()

# End timing
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.4f} seconds")

In [ ]:
#################################################################
### HISTOGRAM COMPARISON
# create a scene, put the seed mesh into the scene
scene2 = o3d.t.geometry.RaycastingScene()
_ = scene2.add_triangles(mesh)

#calculate distance from dust to seed mesh
distance_dust_adjust = scene2.compute_distance(new_dust_points).numpy()
#print(distance_dust_adjust)
hist3, bins3 = np.histogram(distance_dust_adjust, bins=bins_mm)

distance_dust_adjust = distance_dust_adjust*1000
mean_dist_array_lichen = np.array(mean_dist_array_lichen)*1000

# Define histograms for the sampling and laser scanning
hist_adjust_dust, _ = np.histogram(distance_dust_adjust, bins = bins_mm*1000, density = True)
hist_lichen, _ = np.histogram(mean_dist_array_lichen, bins = bins_mm*1000, density = True)

# calculate the expected dust accumulation (f_exp) for the chi-squared test 
# removes the last two bins due to aberrant empty bins. 
expected_dust = len(distance_dust_adjust) * hist_lichen[:-2] * (1/sum(hist_lichen[:-2])) # latter term is ~1.25 and adjusts for 
                                                                                    # I believe the bin width being 80% of 1 cm

# chi squared result comparing the dust accumulation to the laser scan distribution of Ramalina siliquosa
chisquare(f_obs = hist3[:-2], f_exp = expected_dust, ddof=0, axis=0)

# now plot the laser scan lichen distribution as a probability density histogram along with
# the dust accumulation

fig = plt.figure(figsize=(16, 7)) 
gs1 = plt.subplot2grid((1, 2), (0, 0))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]

for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.set_aspect('equal')

fig.subplots_adjust(wspace=0.05, hspace=0.05)

ax1 = plt.subplot2grid((1, 2), (0, 0))
ax2 = plt.subplot2grid((1, 2), (0, 1))

ax1.hist(abs(np.asarray(distance_dust_adjust)), density = True, color= [(0, 114/255.0, 178/255.0)], edgecolor='black', linewidth=1.2, bins=bins_mm*1000, label = 'Dust accumulation per point distance from F602 seed mesh')
ax2.hist(abs(np.asarray(mean_dist_array_lichen)), density = True, color= [(0, 158/255.0, 115/255.0)], edgecolor='black', linewidth=1.2, bins=bins_mm*1000, label = 'Laser scan per point distance of Ramalina siliquosa from rock surface')

textfontsize = 20
ticksize = 16
    
ax1.set_ylabel('Probability density', fontsize=textfontsize)
ax1.set_xlabel('Distance (mm)', fontsize=textfontsize)
ax2.set_xlabel('Distance (mm)', fontsize=textfontsize)

ax1.tick_params(axis='both', which='major', labelsize=ticksize)
ax2.tick_params(axis='both', which='major', labelsize=ticksize)

ax1.tick_params(direction='out', length=6, width=1.5, colors='k',
               grid_color='k', grid_alpha=1.0)
ax2.tick_params(direction='out', length=6, width=1.5, colors='k',
               grid_color='k', grid_alpha=1.0)

ax2.yaxis.set_tick_params(labelleft=False)

lines_array = []
labels_array = []
  
lines, labels = ax1.get_legend_handles_labels()
lines_array.extend(lines)
labels_array.extend(labels)
lines, labels = ax2.get_legend_handles_labels()
lines_array.extend(lines)
labels_array.extend(labels)

fig.legend(lines_array, labels_array, bbox_to_anchor=(0.675,0.97), loc="upper right",
                          bbox_transform=plt.gcf().transFigure, fontsize=textfontsize)
plt.subplots_adjust(left=0.0, bottom=0.1, top=0.8)
plt.savefig(r'***********************.png', 
            dpi= 1000, bbox_inches = 'tight')